<a href="https://colab.research.google.com/github/Meridor6919/BeeClassification/blob/master/BeeClasificationNewModelArchitecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

from tensorflow.keras import layers, models
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

In [2]:
dataset = tfds.load('bee_dataset', split=['train'], as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/7490 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/bee_dataset/bee_dataset_300/1.0.0.incompleteMEA5MP/bee_dataset-train.tfrecord*…

Dataset bee_dataset downloaded and prepared to ~/tensorflow_datasets/bee_dataset/bee_dataset_300/1.0.0. Subsequent calls will reuse this data.


In [3]:
#Split between training data, validation data, test data
splitSize = (0.0, 0.765, 0.9, 1.0)

Testing precision on red channel only

In [4]:
array = np.vstack(tfds.as_numpy(dataset[0]))

ranges = list([int(array.shape[0] * x) for x in splitSize])
def getDataset(ranges, idx):
  return ( 
      np.array(list(map(lambda x: x[0][:, :, 0], array[ranges[idx] : ranges[idx+1]]))),
      np.array(list(map(lambda x: x[1]['varroa_output'], array[ranges[idx] : ranges[idx+1]])))
      )

train_images, train_labels = getDataset(ranges, 0)
validation_images, validation_labels = getDataset(ranges, 1)
test_images, test_labels = getDataset(ranges, 2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/numpy/core/shape_base.py:121: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ary = asanyarray(ary)


In [26]:
model = models.Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ])

inputs = tf.keras.Input(shape=(300, 150, 1))
layer = layers.Rescaling(1./255.)(inputs)
layer = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(layer)
layer = layers.MaxPooling2D(pool_size=2)(layer)
layer = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(layer)
layer = layers.MaxPooling2D(pool_size=2)(layer)
layer = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(layer)
layer = layers.MaxPooling2D(pool_size=2)(layer)
layer = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(layer)
layer = layers.Flatten()(layer)
layer = layers.Dropout(0.5)(layer)
layer = layers.Dense(units=64, activation="relu")(layer)
outputs = layers.Dense(1, activation="relu")(layer)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [27]:
model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 300, 150, 1)]     0         
                                                                 
 conv2d_34 (Conv2D)          (None, 298, 148, 32)      320       
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 149, 74, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_35 (Conv2D)          (None, 147, 72, 64)       18496     
                                                                 
 max_pooling2d_26 (MaxPoolin  (None, 73, 36, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_36 (Conv2D)          (None, 71, 34, 128)       7385

In [28]:
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, validation_data=(validation_images, validation_labels), epochs=5)

Epoch 1/5
128/180 [====================>.........] - ETA: 2s - loss: nan - accuracy: 0.7744

KeyboardInterrupt: ignored